In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

In [18]:
data = np.loadtxt("naive.csv",delimiter= ",")
N,n = data.shape
X = data[:,0:n-1]
y = data[:,n-1]
for i in range(len(y)):
    if y[i] == -1:
        y[i] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state = 0)

In [19]:
class NaiveClassifier():
    
    def __init__(self,X_train, X_test, y_train, y_test ):
        X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test
       
    
    def ClassList(self):
        classes = set(y_train)
        classes_list_ = list(classes)
        no_of_class = len(classes)
        return classes_list_,no_of_class

    def Attributes(self):
        A_ = []      # list of all the attributes
        for i in range(X_train.shape[1]):
            #temp = X_train[:,i]
            temp = X_train[:,i]
            A_.append(temp)

        return A_

    def AttributesValues(self):
        A_ = NaiveClassifier.Attributes(self)
        
        list_of_values_in_each_attr_A_ = [] 
        for i in range(len(A_)):
            temp = list( set(A_[i]) )
            list_of_values_in_each_attr_A_.append(temp)
            B_ = list_of_values_in_each_attr_A_
        return B_

    def PosNegCount(self):
        classes_list_,no_of_class = NaiveClassifier.ClassList(self)
        A_ = NaiveClassifier.Attributes(self)
        B_ = NaiveClassifier.AttributesValues(self)
        
        pos_count_list = [] # positive count
        neg_count_list = [] # negative count
        tst1 = []
        for a in range(len(A_)):
           # print("\n \t   A_{} \n".format(a+1))
            d1_temp = []
            d2_temp = []
            tst = []
            for b in B_[a]:
                q = [i for i,val in enumerate(A_[a]) if val == b] # find the index of values that the attribute can take
                yy = []                                      #
                for i in q:                          #
                    yy.append(y_train[i]) # find the values at those index from 'y_train'

               # print("   pos \t"+ "  \t  neg " )

                d1 = [ b,yy.count(classes_list_[1]) ]
                d1_temp.append(d1)

                d2 = [ b,yy.count(classes_list_[0]) ]
                d2_temp.append(d2)
               # print(d1, " \t",d2)
               # print("\n")
                tst.append(yy)                   # d1 - positive values dictionary
                                                # d2 - negative values dictionary
            tst1.append(tst)
            pos_count_list.append(d1_temp)  # pos_count_list[0] - attribute-1 yes count
            neg_count_list.append(d2_temp)

        return pos_count_list, neg_count_list

    def TotalPositiveCount(self):
        A_ = NaiveClassifier.Attributes(self)
        pos_count_list, neg_count_list = NaiveClassifier.PosNegCount(self)
        
        tot_pos_count_each_attr = [] # for each attribute total positive count
        for a in range(len(A_)):
            t = []
            for i in range(len(pos_count_list[a])):
                t.append(pos_count_list[a][i][1])
            tot_pos_count_each_attr.append(np.sum(t)) # tot_pos_count_each_attr[0] - attribute-1 r total positive count
        return tot_pos_count_each_attr

    def TotalNegativeCount(self):
        A_ = NaiveClassifier.Attributes(self)
        pos_count_list, neg_count_list = NaiveClassifier.PosNegCount(self)
        
        tot_neg_count_each_attr = [] # for each attribute total neagtive count
        for a in range(len(A_)):
            t = []
            for i in range(len(neg_count_list[a])):
                t.append(neg_count_list[a][i][1])
            tot_neg_count_each_attr.append(np.sum(t))
        return tot_neg_count_each_attr

    def pos_probabilities(self):
        A_ = NaiveClassifier.Attributes(self)
        pos_count_list, neg_count_list = NaiveClassifier.PosNegCount(self)  
        tot_pos_count_each_attr = NaiveClassifier.TotalPositiveCount(self)
            
        pos_prob = []
        for k in range(len(A_)):
            prb = []
            for i in range(len(pos_count_list[k])):
                p = pos_count_list[k][i][1] / tot_pos_count_each_attr[k]
                prb.append(p)
            pos_prob.append(prb)
        return pos_prob

    def neg_probabilities(self):
        A_ = NaiveClassifier.Attributes(self)
        pos_count_list, neg_count_list = NaiveClassifier.PosNegCount(self)
        tot_neg_count_each_attr = NaiveClassifier.TotalNegativeCount(self)
        
        neg_prob = []
        for k in range(len(A_)):
            prb = []
            for i in range(len(neg_count_list[k])):
                p = neg_count_list[k][i][1] / tot_neg_count_each_attr[0]
                prb.append(p)
            neg_prob.append(prb)
        return neg_prob

    def ProbailityOf_y(self):
        classes_list_,no_of_class = NaiveClassifier.ClassList(self)
        
        prob_y_ = []
        for i in range(no_of_class):
            tmp = list(y_train).count(classes_list_[i]) / len(y_train)
            prob_y_.append(tmp)
        return prob_y_

    def ProbabiityOf_eachRow(self):
        A_ = NaiveClassifier.Attributes(self)
        pos_prob = NaiveClassifier.pos_probabilities(self)
        neg_prob = NaiveClassifier.neg_probabilities(self)
        pos_count_list, neg_count_list = NaiveClassifier.PosNegCount(self)
        prob_y_ = NaiveClassifier.ProbailityOf_y(self)
        #
        neg_pos_prob_each_row = []
        for q in range(len(X_train)):
            x1 = X_train[q]
            l = []        # for positive
            s = []        # for negative
            for i in range(len(A_)):
                for [a,b],[c,d] in list(zip(pos_count_list[i],neg_count_list[i])):
                    if  a == x1[i] or c == x1[i]:
                        l.append( pos_prob[i][ pos_count_list[i].index([a,b]) ] )
                        s.append( neg_prob[i][ neg_count_list[i].index([c,d]) ] )
                        w =  [np.prod(s)*prob_y_[0],np.prod(l)*prob_y_[1] ] 
            neg_pos_prob_each_row.append(w)
            
        return neg_pos_prob_each_row

    def Predicted_y(self):
        classes_list_,no_of_class = NaiveClassifier.ClassList(self)
        neg_pos_prob_each_row = NaiveClassifier.ProbabiityOf_eachRow(self)
        
        pred_y_ = []
        for i in range(len(y_train)):
            k = neg_pos_prob_each_row[i].index(max(neg_pos_prob_each_row[i]))
            pred_y_.append(classes_list_[k])
        return pred_y_

    def accuracy(self):
        pred_y_ = NaiveClassifier.Predicted_y(self)
        
        correct_classification = 0
        for i in range(len(y_train)):
            if (pred_y_[i] == y_train[i]):
                correct_classification += 1

        per = (correct_classification *100) / len(y_train)
        print("\nThe accuray is found to be : ",per)

In [20]:
model = NaiveClassifier(X_train, X_test, y_train, y_test)

In [21]:
model.accuracy()


The accuray is found to be :  97.6608187134503
